### Flight Time Calculator
Making a flight time calculator for large/international airports.

In [1]:
import numpy as np
import pandas as pd
from math import *
import matplotlib.pyplot as plt

Airport data from [OurAirports.com](https://ourairports.com/data/).

In [2]:
df = pd.read_csv('airports.csv')
df.head(2)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN


In [3]:
df.shape

(61979, 18)

In [4]:
df.type.unique()

array(['heliport', 'small_airport', 'closed', 'seaplane_base',
       'balloonport', 'medium_airport', 'large_airport'], dtype=object)

In [5]:
noneed = ['heliport', 'small_airport', 'closed', 'seaplane_base','balloonport','medium_airport']
for typ in noneed:
    df.drop(df[df.type == typ].index,inplace = True)

In [6]:
df.shape

(617, 18)

In [7]:
df[df.iso_country == 'US'].head(2)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
29476,16091,KABQ,large_airport,Albuquerque International Sunport,35.040199,-106.609001,5355.0,NaN,US,US-NM,Albuquerque,yes,KABQ,ABQ,ABQ,http://www.abqsunport.com/,https://en.wikipedia.org/wiki/Albuquerque_Inte...,NaN
29495,3364,KADW,large_airport,Joint Base Andrews,38.810799,-76.866997,280.0,NaN,US,US-MD,Camp Springs,no,KADW,ADW,ADW,http://www.jba.af.mil/,https://en.wikipedia.org/wiki/Joint_Base_Andrews,Andrews Air Force Base


In [8]:
df[df.iata_code == 'PHL']

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
32052,3771,KPHL,large_airport,Philadelphia International Airport,39.871899,-75.241096,36.0,NaN,US,US-PA,Philadelphia,yes,KPHL,PHL,PHL,http://www.phl.org/,https://en.wikipedia.org/wiki/Philadelphia_Int...,NaN


Great-circle distance given by [Wikipedia: Great-circle distance](https://en.wikipedia.org/wiki/Great-circle_distance).

In [9]:
def cosd(x):
    pi = 3.1415926535
    d2r = pi/180
    return cos(d2r*x)

def sind(x):
    pi = 3.1415926535
    d2r = pi/180
    return sin(d2r*x)

def geodist(x,y):
    '''Input: x, y are 2-tuples (angular spherical coordinates); [Lat, Long]
        Output: Distance between two points on the earth'''
    lat1 = x[0]; lat2 = y[0]; long1 = x[1]; long2 = y[1];
    long_diff = abs(long1 - long2)
    
    r_Earth = 3958.8 # radius of Earth in miles
    cent_ang = acos(sind(lat1)*sind(lat2) + cosd(lat1)*cosd(lat2)*cosd(long_diff))
    return r_Earth*cent_ang
    

In [10]:
def airportDist(x,y):
    '''Function: '''
    iata_codes = df.iata_code.unique()
    if x not in iata_codes:
        raise ValueError(x+' is not a valid IATA Code.')
    
    if y not in iata_codes:
        return ValueError(y+' is not a valid IATA Code.')
    
    lat1 = float(df[df.iata_code == x].latitude_deg)
    long1 = float(df[df.iata_code == x].longitude_deg)
    lat2 = float(df[df.iata_code == y].latitude_deg)
    long2 = float(df[df.iata_code == y].longitude_deg)
    
    loc1 = [lat1, long1]
    loc2 = [lat2, long2]
    
    return geodist(loc1, loc2)
    

In [11]:
# create list of airport names
airport_names = []
for name in df.name:
    airport_names.append(name)

    
def airportCodeLookup(airport):
    for name in airport_names:
        if airport in name:
            return df[df.name == name]['iata_code'].values[0]    

        
def flightTime():
    flightSpeeds = [460, 575]
    depart = input('Enter departure airport code: ')
    destin = input('Enter destination airport code: ')
    
    distance = airportDist(depart,destin)
    times = []
    for speed in flightSpeeds:
        times.append(distance/speed)
    
    return 'Flight time between ' + str(times[1]) + ' and '+ str(times[0]) + ' hours.'
    

In [12]:
airportCodeLookup('Los Angeles')

'LAX'

In [13]:
flightTime()

Enter departure airport code:  PHL
Enter destination airport code:  DEL


'Flight time between 12.843985276584604 and 16.054981595730755 hours.'